In [3]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, time

In [ ]:
attraction_path = 'rawdata/서울시 관광 명소.csv'
cctv_path = 'rawdata/서울시 불법주정차_전용차로 위반 단속 CCTV 위치정보.csv'

df_attraction = pd.read_csv(attraction_path, encoding='cp949')
df_cctv = pd.read_csv(cctv_path, encoding='cp949')

FileNotFoundError: [Errno 2] No such file or directory: '../rawdata/서울시 관광 명소.csv'

In [ ]:
print(df_attraction.shape)
df_attraction.head()

(2213, 15)


,고유번호,언어,상호명,콘텐츠URL,주소,신주소,전화번호,팩스번호,웹사이트,운영시간,운영요일,휴무일,교통정보,태그,장애인편의시설
0,46800,zh-CN,2GIL29 GALLERY,https://chinese.visitseoul.net/attractions/202...,서울 강남구 신사동 519-22,06034 首?江南?江南大路 158街 35 2GIL29 GALLERY,+82-2-6203-2015,NaN,https://www.2gil29gallery.com/,10:00~19:00,周二至周六,周日、周一定期歇? 9/16~18中秋?期????休假,距쎇3??、新盆唐?新沙站8?出口?460m(步行6分?),"江南,新沙,江南展?,?廊,美??,展?",NaN
1,46802,zh-TW,2GIL29 GALLERY,https://tchinese.visitseoul.net/attractions/20...,서울 강남구 신사동 519-22,06034 首爾江南區江南大路158街35號 2GIL29 GALLERY,+82-2-6203-2015,NaN,https://www.2gil29gallery.com/,10:00~19:00,週二~週六,週日以及週一固定休館 9/16~9/18 中秋假期休館,距離3號線、新盆唐線新沙站8號出口約460公尺 (步行6分鐘),"GALLERY,新沙,江南,展覽,美術館,江南展覽",NaN
2,42400,ja,イェンナルチプナグォン(?園)アグチム,https://japanese.visitseoul.net/attractions/百年...,,03133 ソウル市チョンノ(鐘路)?サミルデロ436(ナグォンドン(?園洞)),+82-2-741-3621,NaN,NaN,11:30-22:00(KST),NaN,NaN,地下?1?線/3?線/5?線「チョンノサムガ」?5番出口から約167m (徒? 2分),"百年の店,海鮮の蒸し物,アンコウの蒸し物,イェンナルチプナグォン(?園)アグチム,海鮮鍋,ア...",NaN
3,46799,ja,2GIL29 GALLERY,https://japanese.visitseoul.net/attractions/20...,서울 강남구 신사동 519-22,06034 ソウル市カンナム(江南)?カンナムデロ158ギル35,+82-2-6203-2015,NaN,https://www.2gil29gallery.com/,10:00-19:00,火曜日?土曜日,日曜日、月曜日は定休日 9月16日?18日まで、チュソク(秋夕:??の盆休み)連休のため休み,地下?3?線/シンブンダン線「シンサ」?8番出口から約460m（徒?6分）,"カンナム,美術館,展示,カンナム展示,江南展示,ギャラリ?,神社,江南",NaN
4,46797,en,2Gil29Gallery,https://english.visitseoul.net/attractions/202...,서울 강남구 신사동 519-22,"06034 35 Gangnam-daero 158-gil, Gangnam-gu, Se...",+82-2-6203-2015,NaN,https://www.2gil29gallery.com/,10:00-19:00,Tue-Sat,Closed on Sundays and Mondays Closed for Chu...,"Subway Line 3 or Shinbundang Line, Sinsa Stati...","Shrine,Exhibition,ArtMuseum,Gangnam Exhibition...",NaN


In [ ]:
# Keep only English entries and select only '언어' and '주소' columns
df_attraction = df_attraction[df_attraction['언어'] == 'en'][['언어', '주소']]
df_attraction

,언어,주소
4,en,서울 강남구 신사동 519-22
5,en,"414-10, Jeongneung-dong, Seongbuk-gu, Seoul, ..."
6,en,서울 종로구 창신동 23-322
8,en,서울 종로구 통의동 32
11,en,"33, Hyoja-ro, Jongno-gu, Seoul"
...,...,...
2136,en,"135-897 651-24, Sillim-dong, Gwanak-gu, Seoul"
2137,en,"182-2, Gwanhun-dong, Jongno-gu, Seoul"
2139,en,"136-823 Seongbuk-dong 330-577, Seongbuk-gu, Se..."
2141,en,"32, Seorae-ro, Banpo 4(sa)-dong, Seocho-gu, S..."


In [ ]:
# Function to get coordinates for a given address
def get_coordinates(address):
    apiurl = "https://api.vworld.kr/req/address?"
    apikey =  "D9C1CFC9-0FD3-3304-A3E7-131C33A91190"

    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "parcel",
        "key": apikey
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            point_info = data['response']['result']['point']
            return point_info['x'], point_info['y']
    return None, None


In [ ]:
# Geocode missing coordinates
df_attraction[['경도', '위도']] = df_attraction['주소'].apply(lambda addr: pd.Series(get_coordinates(addr)))

# Display the updated dataframe
df_attraction